In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torchvision.transforms as T
from torchvision import datasets, models, transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torch.utils.data import random_split
import copy
import numpy as np
from collections import defaultdict
import os

# Import necessary PyTorch libraries
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler


In [77]:
train_transforms = transforms.Compose([#transforms.ToPILImage(),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       #transforms.RandomVerticalFlip(),
                                       #transforms.RandomRotation(45),
                                       #transforms.RandomAffine(45),
                                       transforms.ColorJitter(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

# Use 10-crop for Test Time Augmentation
test_transforms = transforms.Compose([#transforms.ToPILImage(),
                                      transforms.Resize(256),
                                      transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])


train_data = datasets.ImageFolder("D:/kood", transform=train_transforms)
#D:\food\testdata
test_data = datasets.ImageFolder("D:/food/testdata", transform=test_transforms)

In [74]:
batch_size=16

In [78]:
train_dl = DataLoader(train_data, batch_size, shuffle=True)
val_dl = DataLoader(test_data, batch_size, shuffle=False)


In [79]:
# Analyzing the shape of one batch
train_images, train_labels = next(iter(train_dl))
test_images, test_labels = next(iter(val_dl))

print('Shape of first batch of training images: ', train_images.shape)
print('Shape of first batch of test images: ', test_images.shape) # 5-D array because of 10 crops


Shape of first batch of training images:  torch.Size([16, 3, 224, 224])
Shape of first batch of test images:  torch.Size([16, 3, 224, 224])


In [80]:

print("Loaded {} images under {}".format(len(train_data), "train"))
print("Loaded {} images under {}".format(len(test_data), "val"))
    
print("Classes: ")
print(train_data.classes)

Loaded 197076 images under train
Loaded 1563 images under val
Classes: 
['가지볶음', '간장게장', '갈비구이', '갈비찜', '갈비탕', '갈치구이', '갈치조림', '감자전', '감자조림', '감자채볶음', '감자탕', '갓김치', '건새우볶음', '경단', '계란국', '계란말이', '계란찜', '계란후라이', '고등어구이', '고등어조림', '고사리나물', '고추장진미채볶음', '고추튀김', '곰탕_설렁탕', '곱창구이', '곱창전골', '과메기', '굴', '김밥', '김치볶음밥', '김치전', '김치찌개', '김치찜', '깍두기', '깻잎장아찌', '꼬막찜', '꽁치조림', '꽈리고추무침', '꿀떡', '나박김치', '나초', '누룽지', '닭갈비', '닭계장', '닭볶음탕', '당근케이크', '더덕구이', '도넛', '도라지무침', '도토리묵', '돈까스tmp', '동그랑땡', '동태찌개', '돼지고기', '된장국', '된장찌개', '두부김치', '두부조림', '디저트', '딸기케이크', '땅콩조림', '떡갈비', '떡국_만두국', '떡꼬치', '떡볶이', '라면', '라볶이', '레드케이크', '리조또', '마늘빵', '마카롱', '막국수', '만두', '매운탕', '멍게', '메추리알장조림', '멸치볶음', '무국', '무생채', '물냉면', '물회', '미역국', '미역줄기볶음', '배추김치', '백김치', '보쌈', '부추김치', '북엇국', '불고기', '브리또', '비빔냉면', '비빔밥', '산낙지', '삼겹살', '삼계탕', '새우볶음밥', '새우튀김', '샌드위치', '샐러드', '생선전', '소세지볶음', '송편', '수육', '수정과', '수제비', '숙주나물', '순대', '순두부찌개', '스테이크', '스테이크2', '스프', '시금치나물', '시래기국', '식혜', '쌀밥tmp', '아이스크림', '알밥', '애플파이', '애호박볶음', '약과', '약식', '양념게

In [7]:
import os
 
path_dir = 'D:\kood'
 
file_list = os.listdir(path_dir)
for i in file_list:
    print('"'+i+'",')


"가지볶음",
"간장게장",
"갈비구이",
"갈비찜",
"갈비탕",
"갈치구이",
"갈치조림",
"감자전",
"감자조림",
"감자채볶음",
"감자탕",
"갓김치",
"건새우볶음",
"경단",
"계란국",
"계란말이",
"계란찜",
"계란후라이",
"고등어구이",
"고등어조림",
"고사리나물",
"고추장진미채볶음",
"고추튀김",
"곰탕_설렁탕",
"곱창구이",
"곱창전골",
"과메기",
"굴",
"김밥",
"김치볶음밥",
"김치전",
"김치찌개",
"김치찜",
"깍두기",
"깻잎장아찌",
"꼬막찜",
"꽁치조림",
"꽈리고추무침",
"꿀떡",
"나박김치",
"나초",
"누룽지",
"닭갈비",
"닭계장",
"닭볶음탕",
"당근케이크",
"더덕구이",
"도넛",
"도라지무침",
"도토리묵",
"돈까스tmp",
"동그랑땡",
"동태찌개",
"돼지고기",
"된장국",
"된장찌개",
"두부김치",
"두부조림",
"디저트",
"딸기케이크",
"땅콩조림",
"떡갈비",
"떡국_만두국",
"떡꼬치",
"떡볶이",
"라면",
"라볶이",
"레드케이크",
"리조또",
"마늘빵",
"마카롱",
"막국수",
"만두",
"매운탕",
"멍게",
"메추리알장조림",
"멸치볶음",
"무국",
"무생채",
"물냉면",
"물회",
"미역국",
"미역줄기볶음",
"배추김치",
"백김치",
"보쌈",
"부추김치",
"북엇국",
"불고기",
"브리또",
"비빔냉면",
"비빔밥",
"산낙지",
"삼겹살",
"삼계탕",
"새우볶음밥",
"새우튀김",
"샌드위치",
"샐러드",
"생선전",
"소세지볶음",
"송편",
"수육",
"수정과",
"수제비",
"숙주나물",
"순대",
"순두부찌개",
"스테이크",
"스테이크2",
"스프",
"시금치나물",
"시래기국",
"식혜",
"쌀밥tmp",
"아이스크림",
"알밥",
"애플파이",
"애호박볶음",
"약과",
"약식",
"양념게장",
"양념치킨",
"양파링",
"어묵볶음",
"연근조림",
"연어요리",
"열무국수",
"열무김치",
"오믈렛",
"오이소박이"

In [30]:
model = models.resnet101(pretrained=True)            

number_of_features =  model.fc.in_features
model.fc = nn.Linear(number_of_features, 199)


In [31]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [32]:
from torch.optim import lr_scheduler
import time
USE_GPU = True

if USE_GPU:
    model = model.cuda()

In [33]:
criterion = nn.CrossEntropyLoss()

# specify optimizer same as used earlier
optimizer_ft = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# secify scheduler same as used earlier
exp_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft, 'min', patience = 5)

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)



cuda:0


In [35]:
print(len(train_dl.dataset))

197076


In [36]:
dataloaders = {}
dataset_sizes = {}
dataloaders['train'] = train_dl
dataloaders['val'] = val_dl
dataset_sizes['train'] = len(train_dl.dataset)
dataset_sizes['val'] = len(val_dl.dataset)

In [41]:
from torch.autograd import Variable
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:               
                continue
                #model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in tqdm(dataloaders[phase]):
                x, y = data;
                inputs = x.squeeze(0).to(device)
                labels = y.to(device)
#                 if phase == 'train':
#                   inputs, labels_a, labels_b, lam = mixup_data(inputs, labels, 0.4, device)
#                   inputs, targets_a, targets_b = Variable(inputs), Variable(labels_a), Variable(labels_b)
                  
                inputs = inputs.to(device)
#                 if phase == 'train':
#                   labels_a = labels_a.to(device)
#                   labels_b = labels_b.to(device)
#                 else:
#                   labels = labels.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
#                     loss_func = mixup_criterion(labels_a,labels_b,lam)
                    
                    _, preds = torch.max(outputs, 1)

                    loss = criterion(outputs, labels)
#                     if phase=='train':
#                       loss = loss_func(criterion,outputs)
#                     else:
#                         loss = criterion(outputs,labels) 

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
#                 if phase == 'train':
#                   running_corrects += lam * preds.eq(labels_a.data).sum() + (1 - lam) * preds.eq(labels_b.data).sum()
#                 else:
#                     running_corrects += torch.sum(preds == labels.data)


            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == 'train':
                scheduler.step(epoch_loss)            

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            torch.save(model.state_dict(), '10_12_test_199_1.pth')
            # deep copy the model
            #if phase == 'val':
            #    best_acc = epoch_acc
            #    best_model_wts = copy.deepcopy(model.state_dict())
                

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [42]:
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)  

Epoch 0/19
----------



train Loss: 0.4784 Acc: 0.8666

Epoch 1/19
----------



train Loss: 0.4763 Acc: 0.8683

Epoch 2/19
----------



train Loss: 0.4781 Acc: 0.8667

Epoch 3/19
----------



train Loss: 0.4753 Acc: 0.8683

Epoch 4/19
----------


...


train Loss: 0.4784 Acc: 0.8669

Epoch 16/19
----------



train Loss: 0.4791 Acc: 0.8660

Epoch 17/19
----------



train Loss: 0.4783 Acc: 0.8672

Epoch 18/19
----------



train Loss: 0.4737 Acc: 0.8683

Epoch 19/19
----------



train Loss: 0.4803 Acc: 0.8664

Training complete in 3143m 6s
Best val Acc: 0.000000


In [ ]:
model = models.resnet101()            

number_of_features =  model.fc.in_features
model.fc = nn.Linear(number_of_features, 199)

USE_GPU = True
if USE_GPU:
    model = model.cuda()
    
model.load_state_dict(torch.load("10_12_test_199_1.pth"))

In [2]:
model1 = models.resnet101()            

number_of_features =  model1.fc.in_features
model1.fc = nn.Linear(number_of_features, 199)

USE_GPU = True
if USE_GPU:
    model1 = model1.cuda()
    
model1.load_state_dict(torch.load("10_12_test_199_1.pth"))

<All keys matched successfully>

In [4]:
criterion = nn.CrossEntropyLoss()

In [5]:
test_transforms1 = transforms.Compose([transforms.Resize(256),
                                      transforms.TenCrop(224),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(crop) for crop in crops]))])

In [8]:
test_data = datasets.ImageFolder("D:/food/testdata", transform=test_transforms1)
test_loader = torch.utils.data.DataLoader(test_data, batch_size= 16, shuffle = True)

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

    
with open('D:/food/food-101/meta/class.txt', 'r',encoding='UTF8') as txt:
    classes = [l.strip() for l in txt.readlines()]
#classes = ['apple_pie', 'baby_back_ribs', 'baklava']
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(len(classes)))
class_total = list(0. for i in range(len(classes)))
batch_time = AverageMeter('Time', ':6.3f')
losses = AverageMeter('Loss', ':.4e')
top1 = AverageMeter('Acc@1', ':6.2f')
top5 = AverageMeter('Acc@5', ':6.2f')
valid_acc_list = []
#move model to gpu
model1.cuda()
model1.eval()
count =0
# iterate over test data
with torch.no_grad():
  for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if USE_GPU:
      data, target = data.cuda(), target.cuda()
            
      ## For 10-crop Testing
      bs, ncrops, c, h, w = data.size()
      # forward pass: compute predicted outputs by passing inputs to the model
      temp_output = model1(data.view(-1, c, h, w))
      output = temp_output.view(bs, ncrops, -1).mean(1)
      # calculate the batch loss
      loss = criterion(output, target)
      
      acc1, acc5 = accuracy(output, target, topk=(1, 5))
      print()
      #valid_acc_list.append(prec1.data/100)
      

    # update average test loss 
      test_loss += loss.item()*data.size(0)
      # convert output probabilities to predicted class
      top1.update(acc1[0], data.size(0))
        
      top5.update(acc5[0], data.size(0))
    
      _, pred = torch.max(output, 1)    
      # compare predictions to true label
      correct_tensor = pred.eq(target.data.view_as(pred))
      correct = np.squeeze(correct_tensor.numpy()) if not USE_GPU else np.squeeze(correct_tensor.cpu().numpy())
      # calculate test accuracy for each object class
      for i in range(len(target)):
          label = target.data[i]
          class_correct[label] += correct[i].item()
          class_total[label] += 1
            
print(top5.avg)    
# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(len(classes)):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %.2f%% (%2d/%2d)' % (classes[i], 100 * class_correct[i] / class_total[i],
                                                         np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %.2f%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total),
                                                      np.sum(class_correct), np.sum(class_total)))

In [10]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [67]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

    
with open('D:/food/food-101/meta/class.txt', 'r',encoding='UTF8') as txt:
    classes = [l.strip() for l in txt.readlines()]
#classes = ['apple_pie', 'baby_back_ribs', 'baklava']
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(len(classes)))
class_total = list(0. for i in range(len(classes)))
batch_time = AverageMeter('Time', ':6.3f')
losses = AverageMeter('Loss', ':.4e')
top1 = AverageMeter('Acc@1', ':6.2f')
top5 = AverageMeter('Acc@5', ':6.2f')
valid_acc_list = []
#move model to gpu
model1.cuda()
model1.eval()

# iterate over test data
with torch.no_grad():
  for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if USE_GPU:
      data, target = data.cuda(), target.cuda()
            
      ## For 10-crop Testing
      bs, ncrops, c, h, w = data.size()
      # forward pass: compute predicted outputs by passing inputs to the model
      temp_output = model1(data.view(-1, c, h, w))
      output = temp_output.view(bs, ncrops, -1).mean(1)
      # calculate the batch loss
      loss = criterion(output, target)
      
      acc1, acc5 = accuracy(output, target, topk=(1, 5))
      #valid_acc_list.append(prec1.data/100)
      

    # update average test loss 
      test_loss += loss.item()*data.size(0)
      # convert output probabilities to predicted class
      top1.update(acc1[0], data.size(0))
        
      top5.update(acc5[0], data.size(0))
    
      _, pred = torch.max(output, 1)    
      # compare predictions to true label
      correct_tensor = pred.eq(target.data.view_as(pred))
      correct = np.squeeze(correct_tensor.numpy()) if not USE_GPU else np.squeeze(correct_tensor.cpu().numpy())
      # calculate test accuracy for each object class
      for i in range(len(target)):
          label = target.data[i]
          class_correct[label] += correct[i].item()
          class_total[label] += 1
            
print(top5.avg)    
# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(len(classes)):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %.2f%% (%2d/%2d)' % (classes[i], 100 * class_correct[i] / class_total[i],
                                                         np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %.2f%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total),
                                                      np.sum(class_correct), np.sum(class_total)))

tensor(99.7098, device='cuda:0')
Test Loss: 0.157293

Test Accuracy of  가지볶음: 97.50% (975/1000)
Test Accuracy of  간장게장: 97.80% (979/1001)
Test Accuracy of  갈비구이: 94.50% (945/1000)
Test Accuracy of   갈비찜: 88.80% (888/1000)
Test Accuracy of   갈비탕: 92.30% (923/1000)
Test Accuracy of  갈치구이: 97.80% (978/1000)
Test Accuracy of  갈치조림: 89.45% (899/1005)
Test Accuracy of   감자전: 95.10% (952/1001)
Test Accuracy of  감자조림: 97.90% (980/1001)
Test Accuracy of 감자채볶음: 99.80% (998/1000)
Test Accuracy of   감자탕: 93.02% (933/1003)
Test Accuracy of   갓김치: 86.43% (860/995)
Test Accuracy of 건새우볶음: 98.50% (985/1000)
Test Accuracy of    경단: 99.70% (997/1000)
Test Accuracy of   계란국: 96.00% (961/1001)
Test Accuracy of  계란말이: 99.60% (997/1001)
Test Accuracy of   계란찜: 98.40% (987/1003)
Test Accuracy of 계란후라이: 93.31% (935/1002)
Test Accuracy of 고등어구이: 94.50% (945/1000)
Test Accuracy of 고등어조림: 87.20% (872/1000)
Test Accuracy of 고사리나물: 99.20% (992/1000)
Test Accuracy of 고추장진미채볶음: 95.30% (953/1000)
Test Accuracy of  고추

In [69]:
print("Test Accuracy (Overall): 96.77%")

Test Accuracy (Overall): 96.77%


In [14]:
model1 = models.resnet101()            

number_of_features =  model1.fc.in_features
model1.fc = nn.Linear(number_of_features, 199)

    
model1.load_state_dict(torch.load("10_12_test_199_1.pth"))

<All keys matched successfully>

In [15]:
model1.eval()
example_inputs = torch.rand(1, 3, 224, 224)
resnet101_traced = torch.jit.trace(model1, example_inputs = example_inputs)
resnet101_traced.save("resnet101_traced.pt")

In [16]:
output = model1(example_inputs)
print(output)

tensor([[ 1.6683e+00,  1.0028e+00,  1.3743e-01, -5.5724e-01,  2.1161e+00,
          2.4716e-01, -1.6218e+00,  3.1941e+00, -3.6085e-01, -4.5259e-02,
          1.6995e+00, -3.6884e-01, -4.1681e-01,  1.4955e+00,  1.5640e+00,
          1.5341e+00,  1.6497e+00,  1.3310e+00,  2.2599e+00, -2.2023e-01,
          4.1880e-01,  3.5987e-02, -1.5106e+00,  4.1830e+00, -3.9829e+00,
         -3.6844e+00, -1.8753e+00, -2.0102e+00,  2.4859e+00,  3.0422e+00,
          1.8782e+00,  1.4586e+00,  7.4914e-01,  1.4893e+00,  2.6295e-01,
         -1.7380e+00, -1.9766e-01,  8.1723e-01,  1.7849e+00,  7.4463e-01,
         -2.6215e+00,  4.9783e+00,  8.9715e-01,  1.0869e+00, -3.8777e-01,
         -8.3069e-01,  9.8879e-01, -7.0240e-01,  2.6180e-01,  3.5164e+00,
         -1.8721e+00, -1.7326e+00, -2.6319e+00, -2.6202e+00,  1.2930e+00,
          3.2519e-01, -2.0535e+00,  3.2546e-01,  1.2196e+00, -1.1732e+00,
         -4.4355e-01,  2.0602e+00,  1.9116e+00, -1.1772e+00, -5.4464e-01,
          2.7470e+00, -1.5007e+00, -3.

In [13]:
with open('D:/food/food-101/meta/class.txt', 'r',encoding='UTF8') as txt:
    classes = [l.strip() for l in txt.readlines()]

In [9]:
import cv2

from PIL import Image
# list of class names by index, i.e. a name can be accessed like class_names[0]
#class_names = [item[4:] for item in train_data.classes]

def predict_food(img_path):
    
    # load the image and return the predicted breed
    img = Image.open(img_path)
    # Resize
    left_margin = (img.width-224)/2
    bottom_margin = (img.height-224)/2
    right_margin = left_margin + 224
    top_margin = bottom_margin + 224
    img = img.crop((left_margin, bottom_margin, right_margin,   
                      top_margin))
    # Normalize
    img = np.array(img)/255
    mean = np.array([0.485, 0.456, 0.406]) #provided mean
    std = np.array([0.229, 0.224, 0.225]) #provided std
    img = (img - mean)/std
    
    # Move color channels to first dimension as expected by PyTorch
    img = img.transpose((2, 0, 1))
    img = torch.from_numpy(img).type(torch.cuda.FloatTensor) 
    img.unsqueeze_(0)
    ps=torch.exp(model1(img))
    top_p, top_class = ps.topk(1, dim=1)
    
    print(top_p)
    
    print(classes[top_class])
    print(classes[top_class+1])
    print(classes[top_class-1])
    return top_class

In [36]:
def predict_food(img_path):
    
    # load the image and return the predicted breed
    img = Image.open(img_path)
    # Resize
#     left_margin = (img.width-224)/2
#     bottom_margin = (img.height-224)/2
#     right_margin = left_margin + 224
#     top_margin = bottom_margin + 224
#     img = img.crop((left_margin, bottom_margin, right_margin,   
#                       top_margin))
    # Normalize
    img = np.array(img)/255
    mean = np.array([0.485, 0.456, 0.406]) #provided mean
    std = np.array([0.229, 0.224, 0.225]) #provided std
    img = (img - mean)/std
    
    # Move color channels to first dimension as expected by PyTorch
    img = img.transpose((2, 0, 1))
    img = torch.from_numpy(img).type(torch.cuda.FloatTensor) 
    img.unsqueeze_(0)
    ps=torch.exp(model1(img))
    top_p, top_class = ps.topk(7, dim=1)
    
    
    return top_class.data.cpu().numpy()[0]

In [28]:
import os
import random
path_dir = 'D:/food/testdata/qw'
 
file_list = os.listdir(path_dir)


In [29]:
file_list1 = file_list[0:100]

In [30]:
file_list2= file_list[100:200]

In [31]:
file_list3= file_list[200:300]

In [32]:
file_list4= file_list[300:400]

In [33]:
file_list5= file_list[400:500]

In [34]:
count = 0
for k , q in enumerate (file_list1):
    pathtmp = path_dir + "/"+q
    #print(pathtmp)
    tmptt = predict_food(pathtmp)
    
    if classes[tmptt[0]] in q or classes[tmptt[1]] in q or classes[tmptt[2]] in q or classes[tmptt[3]] in q or classes[tmptt[4]] in q or classes[tmptt[5]] in q or classes[tmptt[6]] in q:
        count = count+1
    

   
print(count)



83


In [35]:
count = 0
for k , q in enumerate (file_list2):
    pathtmp = path_dir + "/"+q
    #print(pathtmp)
    tmptt = predict_food(pathtmp)
    
    if classes[tmptt[0]] in q or classes[tmptt[1]] in q or classes[tmptt[2]] in q or classes[tmptt[3]] in q or classes[tmptt[4]] in q or classes[tmptt[5]] in q or classes[tmptt[6]] in q:
        count = count+1
    

   
print(count)

count = 0
for k , q in enumerate (file_list3):
    pathtmp = path_dir + "/"+q
    #print(pathtmp)
    tmptt = predict_food(pathtmp)
    
    if classes[tmptt[0]] in q or classes[tmptt[1]] in q or classes[tmptt[2]] in q or classes[tmptt[3]] in q or classes[tmptt[4]] in q or classes[tmptt[5]] in q or classes[tmptt[6]] in q:
        count = count+1
    

print(count)

count = 0
for k , q in enumerate (file_list4):
    pathtmp = path_dir + "/"+q
    #print(pathtmp)
    tmptt = predict_food(pathtmp)
    
    if classes[tmptt[0]] in q or classes[tmptt[1]] in q or classes[tmptt[2]] in q or classes[tmptt[3]] in q or classes[tmptt[4]] in q or classes[tmptt[5]] in q or classes[tmptt[6]] in q:
        count = count+1
   
print(count)

count = 0
for k , q in enumerate (file_list5):
    pathtmp = path_dir + "/"+q
    #print(pathtmp)
    tmptt = predict_food(pathtmp)
    
    if classes[tmptt[0]] in q or classes[tmptt[1]] in q or classes[tmptt[2]] in q or classes[tmptt[3]] in q or classes[tmptt[4]] in q or classes[tmptt[5]] in q or classes[tmptt[6]] in q:
        count = count+1
    
   
print(count)


81
80
73
86
